In [1]:
import sys
from superneuromat.neuromorphicmodel import NeuromorphicModel
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt




In [2]:
class GraphData():
    def __init__(self, name, config):
        self.name = name
        self.config = config
        self.paper_to_topic = {} # maps the paper ID in the dataset to its topic ID
        self.index_to_paper = []    # creates an index for each paper
        self.topics = []            # the list of topics 
        self.train_papers = []
        self.validation_papers = []
        self.test_papers = []
        self.load_topics()
        self.train_val_test_split()
        self.load_features()
        self.load_graph()


    def load_topics(self):
        if (self.name == "cora"):
            f = open("data/Cora/group-edges.csv", 'r')
            lines = f.readlines()
            f.close()

            for line in lines:
                fields = line.strip().split(",")
                if (fields[1] not in self.topics):
                    self.topics.append(fields[1])
                self.paper_to_topic[fields[0]] = fields[1]
                self.index_to_paper.append(fields[0])
        elif (self.name == "citeseer"):
            f = open("data/citeseer/citeseer.content", 'r')
            lines = f.readlines()
            f.close()

            for line in lines:
                fields = line.strip().split()
                if (fields[-1] not in self.topics):
                    self.topics.append(fields[-1])
                #print(fields[0])
                self.paper_to_topic[fields[0]] = fields[-1]
                self.index_to_paper.append(fields[0])   
        elif (self.name == "pubmed"):
            f = open("data/Pubmed-Diabetes/data/Pubmed-Diabetes.NODE.paper.tab", 'r')
            lines = f.readlines()
            f.close()
            lines = lines[2:]
        
            for line in lines:
                fields = line.strip().split()
                if (fields[1] not in self.topics):
                    self.topics.append(fields[1])
                self.paper_to_topic["paper:"+fields[0]] = fields[1]
                self.index_to_paper.append("paper:"+fields[0])



    def load_features(self):
        self.features = {} # keyed on paper ID, value is the feature vector
        if (self.name == "cora"):
            f = open("data/Cora/cora/cora.content", 'r')
            lines = f.readlines()
            f.close()
            
            for line in lines:
                fields = line.strip().split()
                paper_id = fields[0]
                feature = [int(x) for x in fields[1:-1]]
                self.features[paper_id] = feature
                self.num_features = len(feature)
            print("NUM PAPERS WITH FEATURES: ", len(self.features.keys()))
        elif (self.name == "citeseer"):
            f = open("data/citeseer/citeseer.content", 'r')
            lines = f.readlines()
            f.close()
            for line in lines:
                fields = line.strip().split()
                paper_id = fields[0]
                feature = [int(x) for x in fields[1:-1]]
                self.features[paper_id] = feature
                self.num_features = len(feature)
            print("NUM PAPERS WITH FEATURES: ", len(self.features.keys()))
        elif (self.name == "pubmed"):
            f = open("data/Pubmed-Diabetes/data/Pubmed-Diabetes.NODE.paper.tab", 'r')
            lines = f.readlines()
            f.close()
            feature_line = lines[1]
            fields = feature_line.split()
            fields = fields[1:-1]
            all_features = {}
            for i in range(len(fields)):
                feat = fields[i].split(':')[1]
                all_features[feat] = i

            self.num_features = len(all_features.keys())
            lines = lines[2:]
            for line in lines:
                feature = [0]*self.num_features
                fields = line.split()
                paper_id = "paper:" + fields[0]
                xfields = fields[-1].split('=')
                xfields = xfields[1].split(',')
                for x in xfields:
                    feature[all_features[x]] = 1
                self.features[paper_id] = feature

        self.paper_to_features = {} # keyed on paper ID, value is the number of features it has
        self.feature_to_papers = {} # keyed on feature ID, value is the number of papers that have that feature
        for p in self.features.keys():
            self.paper_to_features[p] = np.sum(self.features[p])
            for i in range(len(self.features[p])):
                if (i not in self.feature_to_papers.keys()):
                    self.feature_to_papers[i] = 0
                self.feature_to_papers[i] += self.features[p][i]


    def load_graph(self):
        if (self.name == "cora"):
            self.graph = nx.read_edgelist("data/Cora/edges.csv", delimiter=",")

        elif (self.name == "citeseer"):
            self.graph = nx.read_edgelist("data/citeseer/citeseer.cites")

        elif (self.name == "pubmed"):
            self.graph = nx.read_edgelist("data/Pubmed-Diabetes/data/edge_list.csv", delimiter=",")


    def train_val_test_split(self):
        np.random.seed(432)
        train_papers = []
        test_papers = []
        validation_papers = []
        check_breakdown = {}
        
        for k in self.topics:
            check_breakdown[k] = 0
        
        while (len(train_papers) < len(self.topics)*20):
            index = np.random.randint(len(self.index_to_paper))
            topic = self.paper_to_topic[self.index_to_paper[index]]
            if (check_breakdown[topic] < 20 and self.index_to_paper[index] not in train_papers):
                train_papers.append(self.index_to_paper[index])
                check_breakdown[topic] += 1
        
        while (len(validation_papers) < len(self.topics)*20):
            index = np.random.randint(len(self.index_to_paper))
            topic = self.paper_to_topic[self.index_to_paper[index]]
            if (check_breakdown[topic] < 40 and self.index_to_paper[index] not in train_papers and self.index_to_paper[index] not in validation_papers):
                validation_papers.append(self.index_to_paper[index])
                check_breakdown[topic] += 1
        
        for i in range(len(self.index_to_paper)):
            if (self.index_to_paper[i] not in train_papers and self.index_to_paper[i] not in validation_papers and self.index_to_paper[i] in self.paper_to_topic.keys()):
                test_papers.append(self.index_to_paper[i])

        self.train_papers = train_papers
        self.test_papers = test_papers
        self.validation_papers = validation_papers 


In [3]:
def load_network(graph):
    # Read paper to paper edge list
    topic_neurons = {}
    # Create paper neurons
    paper_neurons = {}
    i = 0
    for node in graph.graph.nodes:
        if (node not in graph.paper_to_topic.keys()):
            continue
        neuron = model.create_neuron(threshold=1, leak=100000, refractory_period=1000)
        paper_neurons[node] = neuron
        
        
    for t in graph.topics:
        neuron = model.create_neuron(threshold=1, leak=100000, refractory_period=1000)
        topic_neurons[t] = neuron

    print(i)
    for edge in graph.graph.edges:
        if (edge[0] not in graph.paper_to_topic.keys() or edge[1] not in graph.paper_to_topic.keys()):
            continue
        pre = paper_neurons[edge[0]]
        post = paper_neurons[edge[1]]
        graph_weight = 100.0
        graph_delay = 1
        model.create_synapse(pre, post, weight=graph_weight, delay=graph_delay)
        model.create_synapse(post, pre, weight=graph_weight, delay=graph_delay)

    for paper in graph.train_papers:
        paper_neuron = paper_neurons[paper]
        topic_neuron = topic_neurons[graph.paper_to_topic[paper]]
        train_to_topic_w = 1.0
        train_to_topic_d = 1
        model.create_synapse(paper_neuron, topic_neuron, weight=train_to_topic_w, delay=train_to_topic_d)
        model.create_synapse(topic_neuron, paper_neuron, weight=train_to_topic_w, delay=train_to_topic_d)

    for paper in graph.validation_papers:
        for topic in graph.topics:
            paper_neuron = paper_neurons[paper]
            topic_neuron = topic_neurons[topic]
            validation_to_topic_w = 0.001
            validation_to_topic_d = 1
            model.create_synapse(paper_neuron, topic_neuron, enable_stdp=True, weight=validation_to_topic_w, delay=validation_to_topic_d)
            model.create_synapse(topic_neuron, paper_neuron, enable_stdp=True, weight=validation_to_topic_w, delay=validation_to_topic_d)

    for paper in graph.test_papers:
        for topic in graph.topics:
            paper_neuron = paper_neurons[paper]
            topic_neuron = topic_neurons[topic]
            test_to_topic_w = 0.001
            test_to_topic_d = 1
            model.create_synapse(paper_neuron, topic_neuron, enable_stdp=True, weight=test_to_topic_w, delay=test_to_topic_d)
            model.create_synapse(topic_neuron, paper_neuron, enable_stdp=True, weight=test_to_topic_w, delay=test_to_topic_d)


    model.stdp_setup(time_steps=1, Apos=[0.001], Aneg=[0.000001], negative_update=True, positive_update=True)
    
    return paper_neurons, topic_neurons

            

In [29]:
def test_paper(id, graph):
    paper_neurons, topic_neurons = load_network(graph)
    model.add_spike(0, paper_neurons[paper], 100.0)
    model.setup()
    model.simulate(time_steps=7)
    model.print_spike_train()
    topic_ids = {} 
    times = []
    min_weight = 1000
    
    paper

    # Analyze the weights between the test paper neuron and topic neurons
    min_weight = float('inf')
    min_topic = None
    for topic in topic_neurons:
        # Find the synapse from topic neuron to test paper neuron
        synapse_indices = [
            i for i, (pre, post) in enumerate(zip(model.pre_synaptic_neuron_ids, model.post_synaptic_neuron_ids))
            if pre == topic_id and post == test_paper_id
        ]
        if synapse_indices:
            idx = synapse_indices[0]
            weight = model.synaptic_weights[idx]
            print(f"Topic: {topic}, Paper: {paper}, Weight: {weight}")
            if weight < min_weight:
                min_weight = weight
                min_topic = topic
                
                
                #need to setup a list of zipped synapses so I can find the ids and get the weight
                #check above where I setup synapses to get the lists to iterate over

In [30]:
graph = GraphData("citeseer", 0)

NUM PAPERS WITH FEATURES:  3312


In [31]:

model = NeuromorphicModel()
for paper in graph.validation_papers:
    test_paper(paper, graph)
    break


0
Time: 0, Spikes: [0 0 0 ... 0 0 0]
Time: 1, Spikes: [0 0 0 ... 0 0 0]
Time: 2, Spikes: [0 0 0 ... 0 0 0]
Time: 3, Spikes: [0 0 0 ... 0 0 0]
Time: 4, Spikes: [0 0 0 ... 0 0 0]
Time: 5, Spikes: [0 0 0 ... 0 0 0]
Time: 6, Spikes: [0 0 0 ... 0 0 0]
Agents
